Copyright 2023 Google LLC.

SPDX-License-Identifier: Apache-2.0


In [ ]:
#@title License
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# **OneRoster Integration Conformance Tests**


Self link: [Run in Colab](https://colab.research.google.com/github/googleworkspace/oneroster-integration-conformance-tests/blob/main/oneroster_1_1_test_suite.ipynb)![colab](https://cloud.google.com/ml-engine/images/colab-logo-32px.png)

# **Before you begin**

While you are familiarizing yourself with the tests, it might be helpful to run each cell individually. However, when you are ready to submit results to Google, it is preferable to run all of the tests at once for a fully generated test report. Both Colab and Jupyter notebook allow you to run all cells.

The report generated will include a report_card table that will hold the results of each test and a latency_report table that will hold the latency data for each API call. These are the tables that should be submitted to classroom-sis-external@google.com after all the tests are run.

## Setup

Before you run the tests, you will need to include the following in the “Enter credentials” code block:

* Token url to retrieve OAuth 2 credentials
* One Roster URL ending in "/ims/oneroster/v1p1"
* Client ID
* Client secret

You will need to enter the following in the "GetAllTeachers with email filter" code block:

* A teacher email address

# **Basic Grade Sync** [Required]

For best experience, run all at once using ctrl+F9 or `run all` from runtime menu.

In [ ]:
#@title Enter Credentials
token_url = input("Enter token URL: ")
one_roster_url = input(
    "Enter OneRoster URL (ending in /ims/oneroster/v1p1): ")
client_id = input("Enter your client id: ")
client_secret = input("Enter your client secret: ")

In [ ]:
#@title Setup
from datetime import datetime, timedelta
import json
import sys
import uuid
import requests
from tabulate import tabulate

report_card = {}
latency_report = {}
test_headers = ["Test", "Result"]
test_success = "pass"
test_fail = "FAIL"

In [ ]:
#@title Get OAuth Token
payload = "grant_type=client_credentials"
headers = {"Content-Type": "application/x-www-form-urlencoded"}
auth = requests.auth.HTTPBasicAuth(client_id, client_secret)

response = requests.request(
    "POST", token_url, auth=auth, headers=headers, data=payload
)
latency_report["Get OAuth Token"] = response.elapsed.microseconds / 1000

tests = []
if response.status_code == 200:
  oauth_token = response.json()["access_token"]
  request_headers = {"Authorization": "Bearer " + oauth_token}
  put_request_headers = {
      "Content-Type": "application/json",
      "Authorization": "Bearer " + oauth_token,
  }

  print(f"OAuth token retrieved: {oauth_token}\n")
  tests.append(("Status code 200", test_success))
else:
  print(json.dumps(response.json(), indent=2))
  sys.exit()


print(tabulate(tests, headers=test_headers))

report_card["Get OAuth Token"] = tests

In [ ]:
#@title GetAllTeachers
url = one_roster_url + "/teachers?limit=1"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetAllTeachers"] = response.elapsed.microseconds / 1000

tests = []
if response.status_code == 200:
  tests.append(("Status code 200", test_success))
else:
  tests.append(("Status code 200", test_fail))
  print(json.dumps(response.json(), indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetAllTeachers"] = tests

In [ ]:
#@title GetAllTeachers with email filter
teacher_email_address = ""  # @param {type:"string"}
url = f"{one_roster_url}/teachers?filter=email='{teacher_email_address}'&limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetAllTeachers w/ email filter"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = []
if response.status_code == 200:
  tests.append(("Status code 200", test_success))

  if len(data["users"]) == 1 and data["users"][0].keys() >= {
      "email",
      "sourcedId",
  }:
    teacher_sourced_id = data["users"][0]["sourcedId"]
    tests.append(("Validate teacher", test_success))
  else:
    tests.append(("Validate teacher", test_fail))
else:
  tests.append(("Status code 200", test_fail))
  print(json.dumps(data, indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetAllTeachers w/ email filter"] = tests

In [ ]:
#@title GetClassesForTeacher
url = f"{one_roster_url}/teachers/{teacher_sourced_id}/classes?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetClassesForTeacher"] = response.elapsed.microseconds / 1000
data = response.json()

tests = []
if response.status_code == 200:
  class_sourced_id = data["classes"][0]["sourcedId"]
  tests.append(("Status code 200", test_success))
else:
  tests.append(("Status code 200", test_fail))
  print(json.dumps(data, indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetClassesForTeacher"] = tests

In [ ]:
#@title GetStudentsForClass
url = f"{one_roster_url}/classes/{class_sourced_id}/students?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetStudentsForClass"] = response.elapsed.microseconds / 1000
data = response.json()

tests = []
if response.status_code == 200:
  tests.append(("Status code 200", test_success))

  if data["users"][0].keys() >= {"email", "sourcedId"}:
    student_sourced_id = data["users"][0]["sourcedId"]
    tests.append(("Validate student", test_success))
  else:
    tests.append(("Validate student", test_fail))
else:
  tests.append(("Status code 200", test_fail))
  print(json.dumps(data, indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetStudentsForClass"] = tests

In [ ]:
#@title PutLineItem Create
line_item_sourced_id = str(uuid.uuid4())
now = datetime.utcnow()
assign_date = now.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
due_date = (now + timedelta(minutes=1)).strftime("%Y-%m-%dT%H:%M:%S.%f")[
    :-3
] + "Z"

url = f"{one_roster_url}/lineItems/{line_item_sourced_id}"

payload = json.dumps(
    {
        "lineItem": {
            "sourcedId": line_item_sourced_id,
            "title": "Google Test Line Item Title",
            "description": "Test Line Item",
            "resultValueMin": 0,
            "resultValueMax": 100,
            "assignDate": assign_date,
            "dueDate": due_date,
            "class": {"sourcedId": class_sourced_id},
        }
    }
)

response = requests.request(
    "PUT", url, headers=put_request_headers, data=payload
)
latency_report["PutLineItem Create"] = response.elapsed.microseconds / 1000

tests = []
if response.status_code == 201:
  tests.append(("Status code 201", test_success))
  print(f"Created line item with sourced ID {line_item_sourced_id}\n")
else:
  tests.append(("Status code 201", test_fail))
  print(json.dumps(response.json(), indent=2))


print(tabulate(tests, headers=test_headers))

report_card["PutLineItem Create"] = tests

In [ ]:
#@title GetLineItemsForClass
url = f"{one_roster_url}/classes/{class_sourced_id}/lineItems?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetLineItemsForClass"] = response.elapsed.microseconds / 1000
data = response.json()

tests = []
if response.status_code == 200:
  tests.append(("Status code 200", test_success))

  if any(sd["sourcedId"] == line_item_sourced_id for sd in data["lineItems"]):
    tests.append(("Get created line item", test_success))
  else:
    tests.append(("Get created line item", test_fail))
else:
  tests.append(("Status code 200", test_fail))
  print(json.dumps(response.json(), indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetLineItemsForClass"] = tests

In [ ]:
#@title PutLineItem Edit
url = f"{one_roster_url}/lineItems/{line_item_sourced_id}"

payload = json.dumps(
    {
        "lineItem": {
            "sourcedId": line_item_sourced_id,
            "title": (
                "[Amended]"
                " Reallllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllly"
                " Long Google Test Line Item Title"
            ),
            "description": "Test Line Item",
            "resultValueMin": 0,
            "resultValueMax": 100,
            "assignDate": assign_date,
            "dueDate": due_date,
            "class": {"sourcedId": class_sourced_id},
        }
    }
)

response = requests.request(
    "PUT", url, headers=put_request_headers, data=payload
)
latency_report["PutLineItem Edit"] = response.elapsed.microseconds / 1000

tests = []
if response.status_code == 200 or response.status_code == 201:
  tests.append(("Status code 200/201", test_success))
else:
  tests.append(("Status code 200/201", test_fail))
  print(json.dumps(response.json(), indent=2))


print(tabulate(tests, headers=test_headers))

if response.status_code == 201:
  print("\nINFO: Modifying existing resource should return status code 200")

report_card["PutLineItem Edit"] = tests

In [ ]:
#@title GetLineItemsForClass (Assignment Title Limit)
url = f"{one_roster_url}/classes/{class_sourced_id}/lineItems?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetLineItemsForClass: Assignment Title Limit"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = []
if response.status_code == 200:
  tests.append(("Status code 200", test_success))

  for d in data["lineItems"]:
    if d["sourcedId"] == line_item_sourced_id:
      if d:
        tests.append(("Line item title limit", len(d["title"])))
      else:
        tests.append(("Line item exists", test_fail))
        print(json.dumps(d, indent=2))
      break
else:
  tests.append(("Status code 200", test_fail))
  print(json.dumps(data, indent=2))


print(tabulate(tests, headers=test_headers))

report_card["Assignment Title Limit"] = tests

In [ ]:
#@title PutResult Create
result_sourced_id = str(uuid.uuid4())

url = f"{one_roster_url}/results/{result_sourced_id}"

payload = json.dumps(
    {
        "result": {
            "sourcedId": result_sourced_id,
            "score": 80,
            "comment": "",
            "scoreStatus": "fully graded",
            "scoreDate": assign_date,
            "lineItem": {"sourcedId": line_item_sourced_id},
            "student": {"sourcedId": student_sourced_id},
        }
    }
)

response = requests.request(
    "PUT", url, headers=put_request_headers, data=payload
)
latency_report["PutResult Create"] = response.elapsed.microseconds / 1000

tests = []
if response.status_code == 201:
  tests.append(("Status code 201", test_success))
  print(f"Created result with sourced ID {result_sourced_id}\n")
else:
  tests.append(("Status code 201", test_fail))
  print(json.dumps(response.json(), indent=2))


print(tabulate(tests, headers=test_headers))

report_card["PutResult Create"] = tests

In [ ]:
#@title GetResultsForLineItem
url = f"{one_roster_url}/classes/{class_sourced_id}/lineItems/{line_item_sourced_id}/results?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetResultsForLineItem"] = response.elapsed.microseconds / 1000
data = response.json()

tests = []
if response.status_code == 200:
  tests.append(("Status code 200", test_success))

  if any(sd["sourcedId"] == result_sourced_id for sd in data["results"]):
    tests.append(("Get created result", test_success))
  else:
    tests.append(("Get created result", test_fail))
else:
  tests.append(("Status code 200", test_fail))
  print(json.dumps(data, indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetResultsForLineItem"] = tests

In [ ]:
#@title PutResult Edit
url = f"{one_roster_url}/results/{result_sourced_id}"

payload = json.dumps(
    {
        "result": {
            "sourcedId": result_sourced_id,
            "score": 300,
            "comment": "",
            "scoreStatus": "fully graded",
            "scoreDate": assign_date,
            "lineItem": {"sourcedId": line_item_sourced_id},
            "student": {"sourcedId": student_sourced_id},
        }
    }
)

response = requests.request(
    "PUT", url, headers=put_request_headers, data=payload
)
latency_report["PutResult Edit"] = response.elapsed.microseconds / 1000

tests = []
if response.status_code == 200 or response.status_code == 201:
  tests.append(("Status code 200/201", test_success))
else:
  tests.append(("Status code 200/201", test_fail))
  print(json.dumps(response.json(), indent=2))


print(tabulate(tests, headers=test_headers))

if response.status_code == 201:
  print("\nINFO: Modifying existing resource should return status code 200")

report_card["PutResult Edit"] = tests

In [ ]:
#@title GetResultsForLineItem (Extra Credit)
url = f"{one_roster_url}/classes/{class_sourced_id}/lineItems/{line_item_sourced_id}/results?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetResultsForLineItem Extra Credit"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = []
if response.status_code == 200:
  tests.append(("Status code 200", test_success))
else:
  tests.append(("Status code 200", test_fail))
  print(json.dumps(data, indent=2))

for r in data["results"]:
  if r["sourcedId"] == result_sourced_id:
    if r["score"] == 300:
      tests.append(("Result extra credit OK", test_success))
    else:
      tests.append(("Result extra credit OK", test_fail))
      print(json.dumps(r, indent=2))
    break


print(tabulate(tests, headers=test_headers))

report_card["Result Extra Credit"] = tests

In [ ]:
#@title DeleteResult
url = f"{one_roster_url}/results/{result_sourced_id}"

response = requests.request("DELETE", url, headers=request_headers)

tests = []
if response.status_code == 200 or response.status_code == 204:
  tests.append(("Status code 200/204", test_success))
  latency_report["DeleteResult"] = response.elapsed.microseconds / 1000
else:
  tests.append(("Status code 200/204", test_fail))
  print(json.dumps(response.json(), indent=2))


print(tabulate(tests, headers=test_headers))

report_card["DeleteResult"] = tests

In [ ]:
#@title GetResultsForLineItem (Deleted)
url = f"{one_roster_url}/classes/{class_sourced_id}/lineItems/{line_item_sourced_id}/results?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetResultsForLineItem (Deleted)"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = []
if not any(sd["sourcedId"] == result_sourced_id for sd in data["results"]):
  tests.append(("Result deleted", test_success))
else:
  tests.append(("Result deleted", test_fail))
  print(json.dumps(data, indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetResultsForLineItem (Deleted)"] = tests

In [ ]:
#@title DeleteLineItem (Optional / Cleanup)
url = f"{one_roster_url}/lineItems/{line_item_sourced_id}"

response = requests.request("DELETE", url, headers=request_headers)
latency_report["DeleteLineItem"] = response.elapsed.microseconds / 1000

tests = []
if response.status_code == 200 or response.status_code == 204:
  tests.append(("Status code 200/204", test_success))
else:
  tests.append(("Status code 200/204", test_fail))
  print(json.dumps(response.json(), indent=2))


print(tabulate(tests, headers=test_headers))

report_card["DeleteLineItem"] = tests

In [ ]:
#@title GetLineItemsForClass (Deleted)
url = f"{one_roster_url}/classes/{class_sourced_id}/lineItems?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetLineItemsForClass (Deleted)"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = []
if not any(sd["sourcedId"] == line_item_sourced_id for sd in data["lineItems"]):
  tests.append(("Line item deleted", test_success))
else:
  tests.append(("Line item deleted", test_fail))
  print(json.dumps(data, indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetLineItemsForClass (Deleted)"] = tests

In [ ]:
#@title Print Report

table_data = []
for test_name, results in report_card.items():
  for result in results:
    table_data.append((test_name, result[0], result[1]))

print(tabulate(table_data, headers=["Test Name", "Result", "Outcome"]))

print("\n")
print(tabulate(latency_report.items(), headers=["Test Name", "Latency (ms)"]))

# **Grading Categories** [Required]

*Only one GET categories endpoint is needed. Please comment out any that are not implemented in order for `run all` to execute successfully.*

In [ ]:
#@title GetAllCategories
url = f"{one_roster_url}/categories?limit=10000"

response = requests.request("GET", url, headers=request_headers, data=payload)
latency_report["GetAllCategories"] = response.elapsed.microseconds / 1000
data = response.json()

tests = []
if response.status_code == 200:
  tests.append(("Status code 200", test_success))
  category_sourced_id = data["categories"][0]["sourcedId"]
else:
  tests.append(("Status code 200", test_fail))
  print(json.dumps(response.json(), indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetAllCategories"] = tests

In [ ]:
#@title GetCategoriesForClass
url = f"{one_roster_url}/classes/{class_sourced_id}/categories?limit=10000"

response = requests.request("GET", url, headers=request_headers, data=payload)
latency_report["GetCategoriesForClass"] = response.elapsed.microseconds / 1000
data = response.json()

tests = []
if response.status_code == 200:
  tests.append(("Status code 200", test_success))
  category_sourced_id = data["categories"][0]["sourcedId"]
else:
  tests.append(("Status code 200", test_fail))
  print(json.dumps(response.json(), indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetCategoriesForClass"] = tests

In [ ]:
#@title PutLineItem w/ category
line_item_sourced_id = str(uuid.uuid4())

url = f"{one_roster_url}/lineItems/{line_item_sourced_id}"

payload = json.dumps(
    {
        "lineItem": {
            "sourcedId": line_item_sourced_id,
            "title": "Google Test Line Item Title",
            "description": "Test Line Item",
            "resultValueMin": 0,
            "resultValueMax": 100,
            "assignDate": assign_date,
            "dueDate": due_date,
            "category": {"sourcedId": category_sourced_id},
            "class": {"sourcedId": class_sourced_id},
        }
    }
)

response = requests.request(
    "PUT", url, headers=put_request_headers, data=payload
)
latency_report["PutLineItem w/ Category"] = response.elapsed.microseconds / 1000

tests = []
if response.status_code == 201:
  tests.append(("Status code 201", test_success))
  print(f"Created line item with sourced ID {line_item_sourced_id}\n")
else:
  tests.append(("Status code 201", test_fail))
  print(json.dumps(response.json(), indent=2))


print(tabulate(tests, headers=test_headers))

report_card["PutLineItem w/ Category"] = tests

In [ ]:
#@title GetLineItemsForClass (Category exists)
url = f"{one_roster_url}/classes/{class_sourced_id}/lineItems?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetLineItemsForClass (Category exists)"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = []

for d in data["lineItems"]:
  if d["sourcedId"] == line_item_sourced_id:
    if d and d["category"]["sourcedId"] == category_sourced_id:
      tests.append(("Line item created with category", test_success))
    else:
      tests.append(("Line item created with category", test_fail))
      print(json.dumps(d, indent=2))
    break


print(tabulate(tests, headers=test_headers))

report_card["GetLineItemsForClass (Category exists)"] = tests

In [ ]:
#@title DeleteLineItem (Optional / Cleanup)
url = f"{one_roster_url}/lineItems/{line_item_sourced_id}"

response = requests.request("DELETE", url, headers=request_headers)

tests = []
if response.status_code == 200 or response.status_code == 204:
  tests.append(("Status code 200/204", test_success))
else:
  tests.append(("Status code 200/204", test_fail))
  print(json.dumps(response.json(), indent=2))


print(tabulate(tests, headers=test_headers))

In [ ]:
#@title Print Report

table_data = []
for test_name, results in report_card.items():
  for result in results:
    table_data.append((test_name, result[0], result[1]))

print(tabulate(table_data, headers=["Test Name", "Result", "Outcome"]))

print("\n")
print(tabulate(latency_report.items(), headers=["Test Name", "Latency (ms)"]))

# **Grading Periods** [Required]

In [ ]:
#@title GetAllClasses w/ filter
url = f"{one_roster_url}/classes?filter=sourcedId='{class_sourced_id}'&limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetAllClasses w/ filter"] = response.elapsed.microseconds / 1000
data = response.json()

tests = []
if response.status_code == 200:
  tests.append(("Status code 200", test_success))

  if len(data["classes"]) == 1 and data["classes"][0].keys() >= {
      "terms",
      "sourcedId",
  }:
    term_sourced_id = data["classes"][0]["terms"][0]["sourcedId"]
    tests.append(("Validate class", test_success))
  else:
    tests.append(("Validate class", test_fail))
else:
  tests.append(("Status code 200", test_fail))
  print(json.dumps(data, indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetAllClasses w/ filter"] = tests

In [ ]:
#@title GetGradingPeriodsForTerm
url = f"{one_roster_url}/terms/{term_sourced_id}/gradingPeriods?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetGradingPeriodsForTerm"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = []
if response.status_code == 200:
  tests.append(("Status code 200", test_success))

  if data["academicSessions"][0].keys() >= {
      "sourcedId",
      "title",
      "startDate",
      "endDate",
      "schoolYear",
  }:
    tests.append(("Validate grading period", test_success))
  else:
    tests.append(("Validate grading period", test_fail))

  tests.append(("Academic sessions all grading period types", test_success))
  for d in data["academicSessions"]:
    if d["type"] != "gradingPeriod":
      tests.append(("Academic sessions all grading period types", test_fail))
      print(json.dumps(d, indent=2))
      break
else:
  tests.append(("Status code 200", test_fail))
  print(json.dumps(data, indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetGradingPeriodsForTerm"] = tests

In [ ]:
#@title Print Report

table_data = []
for test_name, results in report_card.items():
  for result in results:
    table_data.append((test_name, result[0], result[1]))

print(tabulate(table_data, headers=["Test Name", "Result", "Outcome"]))

print("\n")
print(tabulate(latency_report.items(), headers=["Test Name", "Latency (ms)"]))